In [23]:

import wandb
import pandas as pd
import re
from collections import Counter
from tqdm import tqdm

In [18]:
def clean_summary(summary: dict):
    cleaned = {}

    for key, value in summary.items():
        # 1. drop nulls
        if value is None or value == "NaN":
            continue

        # 2. rename keys ending in lowercase 'sari'
        #    ONLY if the suffix is exactly 'sari'
        new_key = re.sub(r'sari$', 'SARI', key)

        cleaned[new_key] = value

    return cleaned

In [19]:

api = wandb.Api()

run = api.run("uds/Thesis_Phase_v3/d71d90d1d998")
print(f"Run name: {run.name}")
print(run.summary)
run.summary.clear()

cleaned_summary = clean_summary(run.summary)
print("Cleaned Summary:")
print(cleaned_summary)


# Upload cleaned summary
run.summary.update(cleaned_summary)

# Force persistence
run.update()

Run name: Shared Baseline
{'_runtime': 32542, '_step': 3569, '_timestamp': 1764109708.628123, '_wandb': {'runtime': 32542}, 'eval/cleaned_baseline_validation_BERTScore_F1': 96, 'eval/cleaned_baseline_validation_SARI': 'NaN', 'eval/cleaned_baseline_validation_dFKGL_SARI': 'NaN', 'eval/cleaned_baseline_validation_label-tgt-dFKGL': 0.25209517045454555, 'eval/cleaned_baseline_validation_pred-label-dFKGL': 1.049662642045455, 'eval/cleaned_baseline_validation_pred-tgt-dFKGL': 1.1697265625, 'eval/cleaned_baseline_validation_runtime': 340.7477, 'eval/cleaned_baseline_validation_samples_per_second': 16.528, 'eval/cleaned_baseline_validation_sari': 76.25467556276125, 'eval/cleaned_baseline_validation_steps_per_second': 0.517, 'eval/cleaned_grade02_validation_BERTScore_F1': 95.32, 'eval/cleaned_grade02_validation_SARI': 'NaN', 'eval/cleaned_grade02_validation_dFKGL_SARI': 'NaN', 'eval/cleaned_grade02_validation_label-tgt-dFKGL': 0.31601562499999997, 'eval/cleaned_grade02_validation_model_preparat

KeyError: 'clear'

In [20]:
import wandb, re

api = wandb.Api()
run = api.run("uds/Thesis_Phase_v3/d71d90d1d998")

summary = run.summary  # SummarySubDict

# 1) delete null keys
to_delete = [k for k, v in summary.items() if v is None or v == "NaN"]
for k in to_delete:
    del summary[k]

# 2) rename keys ending in lowercase 'sari' -> 'SARI'
to_rename = [k for k in list(summary.keys()) if k.endswith("sari")]
for old_k in to_rename:
    new_k = re.sub(r"sari$", "SARI", old_k)

    # If the destination already exists, keep the non-null / latest value
    if new_k not in summary or summary.get(new_k) is None:
        summary[new_k] = summary[old_k]

    del summary[old_k]

# 3) persist
run.update()
print("Updated summary for:", run.path)


Updated summary for: ['uds', 'Thesis_Phase_v3', 'd71d90d1d998']


In [24]:


ENTITY = "uds"
PROJECT = "Thesis_Phase_v3"
DRY_RUN = False  # set True to preview changes without writing

api = wandb.Api()

def fix_run_summary(run, dry_run=False):
    summary = run.summary

    # --- delete null keys ---
    null_keys = [k for k, v in summary.items() if v is None or v == "NaN"]

    # --- rename keys ending with 'sari' ---
    sari_keys = [k for k in list(summary.keys()) if k.endswith("sari")]

    changes = {
        "deleted_nulls": len(null_keys),
        "renamed_sari": 0,
        "skipped_rename_due_to_collision": 0,
    }

    if dry_run:
        # only compute what would happen
        changes["renamed_sari"] = len(sari_keys)
        return changes

    for k in null_keys:
        # guard: key may disappear if W&B lazily loads; try/except is safer
        try:
            del summary[k]
        except KeyError:
            pass

    for old_k in sari_keys:
        try:
            new_k = re.sub(r"sari$", "SARI", old_k)

            # collision handling: prefer existing non-null value, else move old
            if new_k in summary and summary.get(new_k) is not None:
                changes["skipped_rename_due_to_collision"] += 1
                del summary[old_k]
                continue

            summary[new_k] = summary[old_k]
            del summary[old_k]
            changes["renamed_sari"] += 1
        except KeyError:
            # old_k may have disappeared; ignore
            pass

    run.update()
    return changes


runs = api.runs(f"{ENTITY}/{PROJECT}")
print(f"Found {len(runs)} runs in {ENTITY}/{PROJECT}")

totals = Counter()
touched = 0

for run in tqdm(runs):
    ch = fix_run_summary(run, dry_run=DRY_RUN)
    did_something = (ch["deleted_nulls"] > 0) or (ch["renamed_sari"] > 0)
    if did_something:
        touched += 1
        print(f"- {run.id} | {run.name}: "
              f"deleted_nulls={ch['deleted_nulls']}, "
              f"renamed_sari={ch['renamed_sari']}, "
              f"collisions={ch['skipped_rename_due_to_collision']}")
    totals.update(ch)

print("\nSummary")
print(f"Runs touched: {touched}/{len(runs)}")
print(dict(totals))
print("DRY_RUN =", DRY_RUN)


Found 129 runs in uds/Thesis_Phase_v3


  2%|▏         | 2/129 [00:01<01:35,  1.34it/s]

- 353f7c9988d3 | off_the_shelf-graded-eval-6: deleted_nulls=11, renamed_sari=0, collisions=0


  2%|▏         | 3/129 [00:02<01:33,  1.34it/s]

- d143d65a6895 | grade02-from-baseline: deleted_nulls=1, renamed_sari=0, collisions=0


...

- 74125854ca65 | grade12-from-baseline: deleted_nulls=1, renamed_sari=0, collisions=0


 12%|█▏        | 15/129 [00:10<01:19,  1.43it/s]

- ab0b0ea7e364 | dare_ties_g@all_ws@all_w@uniform-avg: deleted_nulls=11, renamed_sari=0, collisions=0


100%|██████████| 129/129 [01:43<00:00,  1.24it/s]


Summary
Runs touched: 14/129
{'deleted_nulls': 44, 'renamed_sari': 0, 'skipped_rename_due_to_collision': 0}
DRY_RUN = False


In [ ]:
ENTITY = "uds"
SRC_PROJECT = "Thesis_Phase_v3.2"
DST_PROJECT = "Thesis_Phase_v3.1"

# Choose which dFKGL variant you want averaged:
DFKGL_VARIANT = "pred-tgt-dFKGL"   # <- change to "label-tgt-dFKGL" or "pred-tgt-dFKGL" if needed

api = wandb.Api()

grade_re = re.compile(r"eval/cleaned_grade(\d+)_validation_(.+)$")

def is_number(x):
    return isinstance(x, (int, float)) and not (isinstance(x, float) and math.isnan(x))

def extract_per_grade(summary_items):
    """
    Returns:
      sari_by_grade: dict[int] -> float
      dfkgl_by_grade: dict[int] -> float  (for the chosen variant)
    """
    sari_by_grade = {}
    dfkgl_by_grade = {}

    for k, v in summary_items:
        if v is None:
            continue

        m = grade_re.match(k)
        if not m:
            continue

        grade = int(m.group(1))
        metric_tail = m.group(2)

        # SARI can appear as ..._sari or ..._SARI
        if metric_tail in ("sari", "SARI") and is_number(v):
            sari_by_grade[grade] = float(v)

        # dFKGL variant (e.g. pred-label-dFKGL)
        if metric_tail == DFKGL_VARIANT and is_number(v):
            dfkgl_by_grade[grade] = float(v)

    return sari_by_grade, dfkgl_by_grade

def avg(vals):
    vals = list(vals)
    return sum(vals) / len(vals) if vals else None

runs = api.runs(f"{ENTITY}/{SRC_PROJECT}")
print(f"Found {len(runs)} runs in {ENTITY}/{SRC_PROJECT}")

created = 0
skipped = 0

for src_run in tqdm(runs):
    summary_items = list(src_run.summary.items())
    sari_by_grade, dfkgl_by_grade = extract_per_grade(summary_items)

    # require at least 1 value in each (you can tighten this to == 11 if you want)
    if not sari_by_grade or not dfkgl_by_grade:
        skipped += 1
        continue

    avg_sari = avg(sari_by_grade.values())
    avg_dfkgl = avg(dfkgl_by_grade.values())

    if avg_sari is None or avg_dfkgl is None:
        skipped += 1
        continue

    # Build a clean summary for the new project
    clean_summary = {
        "avg/SARI": avg_sari,
        f"avg/{DFKGL_VARIANT}": avg_dfkgl,
        "src/run_id": src_run.id,
        "src/run_path": src_run.path,
        "src/name": src_run.name,
    }

    # Also include per-grade metrics with clean, consistent names
    for g, v in sorted(sari_by_grade.items()):
        clean_summary[f"eval/cleaned_grade{g:02d}_validation_SARI"] = v
    for g, v in sorted(dfkgl_by_grade.items()):
        clean_summary[f"eval/cleaned_grade{g:02d}_validation_{DFKGL_VARIANT}"] = v

    # Create a new run (summary-only) in the destination project
    new_run = wandb.init(
        entity=ENTITY,
        project=DST_PROJECT,
        name=src_run.name,
        tags=(list(src_run.tags) if src_run.tags else []) + ["summary_only", "normalized_metrics"],
        config=dict(src_run.config),
        reinit=True,
    )

    wandb.summary.update(clean_summary)
    wandb.finish()
    created += 1

print(f"Done. Created {created} runs. Skipped {skipped} runs.")
print(f"Destination project: {ENTITY}/{DST_PROJECT}")


Found 129 runs in uds/Thesis_Phase_v3


  0%|          | 0/129 [00:00<?, ?it/s]wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


avg/SARI,75.87811
avg/pred-tgt-dFKGL,1.18386
eval/cleaned_grade02_validation_SARI,77.58455
eval/cleaned_grade02_validation_pred-tgt-dFKGL,1.07129
eval/cleaned_grade03_validation_SARI,74.27332
eval/cleaned_grade03_validation_pred-tgt-dFKGL,1.07246
eval/cleaned_grade04_validation_SARI,76.14591
eval/cleaned_grade04_validation_pred-tgt-dFKGL,1.00957
eval/cleaned_grade05_validation_SARI,77.89772
eval/cleaned_grade05_validation_pred-tgt-dFKGL,1.06172
+17,...


  1%|          | 1/129 [00:07<16:37,  7.79s/it]

avg/SARI,33.61873
avg/pred-tgt-dFKGL,3.52775
eval/cleaned_grade02_validation_SARI,35.38935
eval/cleaned_grade02_validation_pred-tgt-dFKGL,4.50508
eval/cleaned_grade03_validation_SARI,34.62463
eval/cleaned_grade03_validation_pred-tgt-dFKGL,4.2627
eval/cleaned_grade04_validation_SARI,35.40479
eval/cleaned_grade04_validation_pred-tgt-dFKGL,3.90059
eval/cleaned_grade05_validation_SARI,35.84795
eval/cleaned_grade05_validation_pred-tgt-dFKGL,3.58906
+17,...


...

avg/SARI,74.37203
avg/pred-tgt-dFKGL,1.19538
eval/cleaned_grade02_validation_SARI,76.53171
eval/cleaned_grade02_validation_pred-tgt-dFKGL,1.02305
eval/cleaned_grade03_validation_SARI,71.28234
eval/cleaned_grade03_validation_pred-tgt-dFKGL,1.15039
eval/cleaned_grade04_validation_SARI,72.9776
eval/cleaned_grade04_validation_pred-tgt-dFKGL,1.08301
eval/cleaned_grade05_validation_SARI,76.71914
eval/cleaned_grade05_validation_pred-tgt-dFKGL,1.0502
+17,...


 99%|█████████▉| 128/129 [06:18<00:02,  2.74s/it]

avg/SARI,26.56557
avg/pred-tgt-dFKGL,19.86385
eval/cleaned_grade02_validation_SARI,24.32357
eval/cleaned_grade02_validation_pred-tgt-dFKGL,16.67402
eval/cleaned_grade03_validation_SARI,22.76431
eval/cleaned_grade03_validation_pred-tgt-dFKGL,17.90742
eval/cleaned_grade04_validation_SARI,28.04166
eval/cleaned_grade04_validation_pred-tgt-dFKGL,16.31445
eval/cleaned_grade05_validation_SARI,20.85524
eval/cleaned_grade05_validation_pred-tgt-dFKGL,20.1334
+17,...


100%|██████████| 129/129 [06:21<00:00,  2.96s/it]

Done. Created 129 runs. Skipped 0 runs.
Destination project: uds/Thesis_Phase_v3.1
